In [ ]:
# default_exp xresnet

# xresnet

> Xresnet models. As in fastai v1, refactored.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# export
import torch.nn as nn
import torch
from collections import OrderedDict

In [ ]:
# export
from model_constructor.constructor import *
from model_constructor.layers import *

# ResBlock

In [ ]:
# export
class ResBlock(nn.Module):
    def __init__(self, ni, nh, expansion=1, stride=1, conv_layer=ConvLayer, 
                 act_fn=act_fn, **kwargs):
        super().__init__()
#         print(f"ni {ni}, nh {nh}, exp {expansion}, stride {stride}")
        nf,ni = nh*expansion,ni*expansion
#         print(f"new ni {ni}, nf {nf}")
        layers  = [('conv_0', conv_layer(ni, nh, 3, stride=stride)),
                   ('conv_1', conv_layer(nh, nf, 3, zero_bn=True, act=False))
        ] if expansion == 1 else [
                   ('conv_0', conv_layer(ni, nh, 1)),
                   ('conv_1', conv_layer(nh, nh, 3, stride=stride)),
                   ('conv_2', conv_layer(nh, nf, 1, zero_bn=True, act=False))
        ]
        self.convs = nn.Sequential(OrderedDict(layers))
        # TODO: check whether act=True works better
        identity = [] if stride==1 else [('pool', nn.AvgPool2d(2, ceil_mode=True))]
        identity += [] if ni==nf else [('idconv', conv_layer(ni, nf, 1, act=False))]
        self.identity = Noop() if identity==[] else nn.Sequential(OrderedDict(identity))
        self.merge = Noop() # us it to visualize in repr residual connection
        self.act_fn = act_fn

    def forward(self, x): return self.act_fn(self.merge(self.convs(x) + self.identity(x)))

In [ ]:
body = Body(ResBlock, expansion=4)
body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 2 blocks, expansion 4.
    (block_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity): Sequential(
        (idconv): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)

In [ ]:
xb = torch.randn(16, 64, 32, 32)
y = body(xb)
y.shape

torch.Size([16, 2048, 4, 4])

# Xresnet models.

In [ ]:
# export
def xresnet18(**kwargs):
    """Constructs a xresnet-18 model. """
    return Net(stem_sizes=[32,32], block=ResBlock, blocks=[2, 2, 2, 2], expansion=1, **kwargs)
def xresnet34(**kwargs):
    """Constructs axresnet-34 model. """
    return Net(stem_sizes=[32,32], block=ResBlock, blocks=[3, 4, 6, 3], expansion=1, **kwargs)
def xresnet50(**kwargs):
    """Constructs axresnet-34 model. """
    return Net(stem_sizes=[32,32],block=ResBlock, blocks=[3, 4, 6, 3], expansion=4, **kwargs)

In [ ]:
Stem()

Stem(
  sizes: [3, 64]
  (conv0): ConvLayer(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
xresnet18()

Net(
  (stem): Stem(
    sizes: [3, 32, 32, 64]
    (conv0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 2 blocks, expansion 1.
      (block_0): ResBlock(
 

In [ ]:
xresnet34()

Net(
  (stem): Stem(
    sizes: [3, 32, 32, 64]
    (conv0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 3 blocks, expansion 1.
      (block_0): ResBlock(
 

In [ ]:
model = xresnet50()

In [ ]:
model.stem

Stem(
  sizes: [3, 32, 32, 64]
  (conv0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
model.body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 3 blocks, expansion 4.
    (block_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity): Sequential(
        (idconv): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)

In [ ]:
model.head

Head(
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flat): Flatten()
  (fc): Linear(in_features=2048, out_features=1000, bias=True)
)

In [ ]:
xb = torch.randn(16, 3, 128, 128)
y = model(xb)
y.shape

torch.Size([16, 1000])

# end. resnet.
model_constructor
by ayasyrev

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_constructor.ipynb.
Converted 01_layers.ipynb.
Converted 02_resnet.ipynb.
Converted 03_xresnet.ipynb.
Converted index.ipynb.
